Loading env required

In [1]:
import scipy.io
# %%
import pickle
from sklearn.utils import shuffle
from datetime import datetime
import sys
import matplotlib.pyplot as plt
import ssm
import autograd.numpy as np
from LapseModel_ntex import lapse_model
from lapse_utils_ntex import get_parmin, get_parmax, get_parstart, fit_lapse_multiple_init, \
    calculate_predictive_acc_lapse_model
from glm_utils_ntex import fit_glm, calculate_predictive_acc_glm, \
    plot_input_vectors, append_zeros, fit_glm_multiple_init
from glm_hmm_utils_ntex import calculate_predictive_acc_glmhmm_ntex_parted, \
    fit_glmhmm_multiple_init, get_posterior_states_labels, get_posterior_states_labels_parted
from plotting_utils_ntex import plot_glmhmm_weights


Load mat file to a python dict

In [2]:
target = r"/mnt/g/My Drive/#Projects/Project_Neurodynamic-Control/GLM_HMM_Data/preprocessed_data/preprocessed_240_241_242_243_Datastore_created_26-Mar-2023"

In [3]:
loaded = scipy.io.loadmat(target)
loaded_mat_keys = loaded.keys()
print(loaded_mat_keys)
input = loaded["preprocessed_input"]
label = loaded["preprocessed_label"]

dict_keys(['__header__', '__version__', '__globals__', 'preprocessed_input', 'preprocessed_label', 'preprocessed_session', 'preprocessed_trial_number', '__function_workspace__'])


1. Due unknown reason, the labels are converted to a weird sahpe tensor wize size: x, 1, y, 1, 1, 1 (which should be x,y,1); Here I made it to correct shape, but not shure if such conversion is stable. Input is also disrrupted to size x,1,y,4 (which should be x,1,y,4), since this is not true tensor( or nd array) due to inconsistent numel in each element (different trial in each session), can not directly use numpy reshape
2. the model only accept labels as type(int, np.int8, np.int16, np.int32, np.int64)

In [4]:
input_compatible = [x[0] for x in input]
label_compatible = [np.array([[y[0][0][0]] for y in x[0]]).astype(int) for x in label ]
#print(input_compatible)

For HMM model,due to its nature of calculating posterior probability, the order of trial is very important. Thus the shuffling of data is in dimension of sessions. Here gives an example of 5-fold model training (iterations of using each 1/5 sessions of data as test set and the other 4/5 sessions as training set), the following part set up all necessary hyperparameters.

In [5]:
input_shuffled, y_shuffled = shuffle(input_compatible, label_compatible,
                                   random_state=66)

# display the shape of input_shuffled
print("input_shuffled shape: ", np.shape(input_shuffled))

input_shuffled shape:  (50,)


In [6]:
input_shuffled

[array([[0.5  , 1.   , 1.   , 1.   ],
        [0.1  , 1.   , 1.   , 1.   ],
        [0.5  , 1.   , 1.   , 1.   ],
        [0.1  , 1.   , 1.   , 1.   ],
        [0.   , 1.   , 1.   , 1.   ],
        [0.5  , 0.   , 0.   , 1.   ],
        [0.025, 1.   , 1.   , 1.   ],
        [0.   , 0.   , 0.   , 1.   ],
        [1.   , 0.   , 0.   , 1.   ],
        [0.5  , 1.   , 1.   , 1.   ],
        [0.1  , 1.   , 1.   , 1.   ],
        [0.   , 1.   , 1.   , 1.   ],
        [0.   , 0.   , 0.   , 1.   ],
        [0.05 , 0.   , 0.   , 1.   ],
        [0.5  , 0.   , 0.   , 1.   ],
        [0.5  , 1.   , 1.   , 1.   ],
        [0.   , 1.   , 1.   , 1.   ],
        [0.025, 0.   , 0.   , 1.   ],
        [0.5  , 0.   , 0.   , 1.   ],
        [0.1  , 1.   , 1.   , 1.   ],
        [0.1  , 1.   , 1.   , 1.   ],
        [0.05 , 1.   , 1.   , 1.   ],
        [0.   , 1.   , 1.   , 1.   ],
        [0.05 , 0.   , 0.   , 1.   ],
        [0.   , 1.   , 1.   , 1.   ],
        [1.   , 0.   , 0.   , 1.   ],
        [1. 

hyperparameters:

In [7]:
fold = 5
C = 2  # number of output types/categories here with only two: response or not
D = 1  # dimension of data (observations)
transition_alpha = 1  # Hyperparameter
prior_sigma = 100  # Hyperparameter
K_states = 3  # Number of states you wish to observe

n_init = 5 # number of times for independent runs in which the one with the best predictive acc would be selected out
N_em_iters = 500  # number of max iterations for the expectation-maximization algorithm, which is set to prevent non-converging situation
global_fit = True  # If global_fit true, use GLM parameter as initial params for glmhmm
# If global_fit false, pretrained glmhmm params are needed and used as as initial params for glmhmm


# get time now and result storing path to saving data such as the trained glmhmm parameters
results_dir = '../results/'
N_property = len(input_shuffled[0][0])
M_GLM = N_property-1  # Number of inputs for each trial for the model
# Since GLM model already has the bias colum, so remove this column before feeding to it, which is column 4
input_index_glm = list(range(0, M_GLM))
M_HMM = N_property  # Number of inputs for each trial for the model
input_index_hmm = list(range(0, M_HMM))

The following part is a for-loop training glm-hmm model on each (fold -1)/fold of data and tested by the other 1/fold:

In [8]:
data_size = len(input_shuffled)
Acc_GLM_5fold = []
Param_GLM_5fold = []
Acc_HMM_5fold = []
Param_HMM_5fold = []

for j in range(fold):
    # get hmm training and test set, for hmm training, the data should be parted into sessions
    input_this_test_hmm = [
        x[:, input_index_hmm]
        for c, x in enumerate(input_shuffled)
        if j*len(input_shuffled)/fold <= c <= (j+1)*len(input_shuffled)/fold]
    y_this_test_hmm = [
        x[:, :]
        for c, x in enumerate(y_shuffled)
        if j*len(input_shuffled)/fold <= c <= (j+1)*len(input_shuffled)/fold]
    input_this_train_hmm = [
        x[:, input_index_hmm]
        for c, x in enumerate(input_shuffled)
        if c < j*len(input_shuffled)/fold or c > (j+1)*len(input_shuffled)/fold]
    y_this_train_hmm = [
        x[:, :]
        for c, x in enumerate(y_shuffled)
        if c < j*len(input_shuffled)/fold or c > (j+1)*len(input_shuffled)/fold]

    # get glm training and test set, for glm training, the data should not be parted, that concatenaed
    input_this_test_glm = []
    input_this_train_glm = []
    y_this_test_glm = []
    y_this_train_glm = []
    for c, x in enumerate(input_shuffled):
        if j*len(input_shuffled)/ fold <= c <= (j + 1)*len(input_shuffled)/ fold:
            if len(input_this_test_glm) == 0:
                input_this_test_glm = x[:, input_index_glm]
                y_this_test_glm = y_shuffled[c]
            else:
                input_this_test_glm = np.concatenate((
                    input_this_test_glm, x[:, input_index_glm]))
                y_this_test_glm = np.concatenate((
                    y_this_test_glm, y_shuffled[c]))
        else:
            if len(input_this_train_glm) == 0:
                input_this_train_glm = x[:, input_index_glm]
                y_this_train_glm = y_shuffled[c]
            else:
                input_this_train_glm = np.concatenate((
                    input_this_train_glm, x[:, input_index_glm]))
                y_this_train_glm = np.concatenate((
                    y_this_train_glm, y_shuffled[c]))

    # run GLM model training
    best_param_glm, best_acc_glm = fit_glm_multiple_init(
        input_this_train_glm, y_this_train_glm,
        input_this_test_glm, y_this_test_glm,
        M_GLM, C, n_init)
    Param_GLM_5fold.append(best_param_glm)
    Param_GLM_5fold.append(best_acc_glm)

    # start GLM HMM training
    now = datetime.now()
    time_str = now.strftime("%Y-%m-%d-%H%M%S")
    weights_glmhmm_example, acc_glmhmm_example = fit_glmhmm_multiple_init(
        input_this_train_hmm, y_this_train_hmm,
        input_this_test_hmm, y_this_test_hmm,
        [np.ones([len(x), 1]) for x in y_this_train_hmm],
        # this parameter provides a function to exclude some trials, if you want to use all trials,
        # create a matrix of ones in the same shape as the y dataset
        K_states, D, M_HMM, C, N_em_iters, transition_alpha,
        prior_sigma, global_fit, best_param_glm,
        'training_cache/glmhmm_' + time_str,
        n_init, partition=True)
    Param_HMM_5fold.append(weights_glmhmm_example)
    Acc_HMM_5fold.append(acc_glmhmm_example)
print(Acc_HMM_5fold)
                                  

=== fitting GLM-HMM ========


  0%|          | 0/500 [00:00<?, ?it/s]

=== fitting GLM-HMM ========


  0%|          | 0/500 [00:00<?, ?it/s]

=== fitting GLM-HMM ========


...

=== fitting GLM-HMM ========


  0%|          | 0/500 [00:00<?, ?it/s]

=== fitting GLM-HMM ========


  0%|          | 0/500 [00:00<?, ?it/s]

[0.8998060762766645, 0.8846694796061885, 0.90067214339059, 0.8993333333333333, 0.876750700280112]


Save best trained model parameters and accuracy for each fold, they could later be used for plotting in python or used as pretrained glmhmm parameters.

In [9]:
#%% save best trained model parameters and accuracy for each fold
time_str = datetime.now().strftime("%Y-%m-%d-%H%M%S")
results_dir = '../results/'
with open(results_dir+'Accs_24x_NEmice_example_' + time_str, 'wb') as f:
    # indent=2 is not needed but makes the file human-readable
    # if the data is nested
    pickle.dump(Acc_HMM_5fold, f)
with open(results_dir+'Params_24x_NEmice_example_' + time_str, 'wb') as f:
    # indent=2 is not needed but makes the file human-readable
    # if the data is nested
    pickle.dump(Param_HMM_5fold, f)


Load the best trained parameter and the example data before shuffling to calculate predicted labels and states of each trial in correct order, which could be saved (in npz format for python env and mat format for matlab env) and provide information for further nalysis

In [11]:
Best_param = Param_HMM_5fold[Acc_HMM_5fold.index(max(Acc_HMM_5fold))]
K_states = 3
#input_compatible
#label_compatible
# states_probs: for each trial what is its probability in state1, state2 and so on
# predicted_states: basically find the max one among the states_probs for each trial
# predicted_response_prob: the predicted probability of response
# predicted_label: If predicted_response_prob > 50, labeled as response trial, otherwise no-response trial
states_probs, predicted_states, predicted_label, predicted_response_prob = \
    get_posterior_states_labels_parted(
        input_compatible, label_compatible,
        Best_param, K_states, range(K_states))

results_dir = '../results/'
time_str =  datetime.now().strftime("%Y-%m-%d-%H%M%S")
np.savez(results_dir+'predicted_states_and_labels_24x_NEmice_' + time_str + '.npz',
         states_probs,
         predicted_states,
         predicted_response_prob,
         predicted_label
         )
result_dir_2matlab = "/mnt/g/My Drive/#Projects/Project_Neurodynamic-Control/GLM_HMM_Data/GLM_HMM_processed_result/"
scipy.io.savemat(result_dir_2matlab+'predicted_states_and_labels_24x_NEmice_Python2mat' + time_str +'.mat',
                 dict(
                     states_probs = states_probs,
                     predicted_states = predicted_states,
                     predicted_response_prob = predicted_response_prob,
                     predicted_label = predicted_label,
                 ))

In [12]:
result_dir_2matlab = "/mnt/g/My Drive/#Projects/Project_Neurodynamic-Control/GLM_HMM_Data/GLM_HMM_processed_result/"
scipy.io.savemat(result_dir_2matlab+'predicted_states_and_labels_24x_NEmice_Python2mat' + time_str +'.mat',
                 dict(
                     states_probs = states_probs,
                     predicted_states = predicted_states,
                     predicted_response_prob = predicted_response_prob,
                     predicted_label = predicted_label,
                 ))